# **Image Processing**

**Import modules**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

**Helper functions**

In [ ]:
def read_img(img_path):
    '''
    Read image from img_path

    Parameters
    ----------
    img_path : str
        Path of image

    Returns
    -------
        Image
    '''

    image = Image.open(img_path)
    return np.array(image)


def show_img(img, colorMap=None):
    '''
    Show image

    Parameters
    ----------
    img : <your type>
        Image
    '''
    
    plt.imshow(img, cmap=colorMap)
    plt.axis('off')
    plt.show()

def save_img(img, img_path):
    '''
    Save image to img_path

    Parameters
    ----------
    img : <your type>
        Image
    img_path : str
        Path of image
    '''

    image = Image.fromarray(img)
    image.save(img_path)


def change_brightness(img, value):
    """
    Change the brightness of the image.

    Parameters
    ----------
    img : numpy.ndarray
        Original image as a numpy array
    value : int
        Value to change the brightness. Positive values to increase brightness, negative to decrease.

    Returns
    -------
    numpy.ndarray
        Image with brightness changed
    """
    
    formula = img.astype(np.int16) + value  
    img_brightened = np.clip(formula, a_min=0, a_max=255).astype(np.uint8)
    
    return img_brightened

def change_contrast(img, factor):
    """
    Change the contrast of the image.

    Parameters
    ----------
    img : numpy.ndarray
        Original image as a numpy array
    factor : float
        Factor to change the contrast. Values > 1 will increase contrast, values between 0 and 1 will decrease contrast.

    Returns
    -------
    numpy.ndarray
        Image with contrast changed
    """
    
    formula = 128 + factor * (img.astype(np.int16) - 128)
    img_contrast = np.clip(formula, a_min=0, a_max=255).astype(np.uint8)

    return img_contrast

def flip_horizontal(img):
    """
    Flip the image horizontally.

    Parameters
    ----------
    img : numpy.ndarray
        Original image as a numpy array

    Returns
    -------
    numpy.ndarray
        Horizontally flipped image
    """
    return np.fliplr(img)

def flip_vertical(img):
    """
    Flip the image vertically.

    Parameters
    ----------
    img : numpy.ndarray
        Original image as a numpy array

    Returns
    -------
    numpy.ndarray
        Vertically flipped image
    """
    return np.flipud(img)

def rgb_to_grayscale(img):
    """
    Convert an RGB image to grayscale.

    Parameters
    ----------
    img : numpy.ndarray
        Original image as a numpy array

    Returns
    -------
    numpy.ndarray
        Grayscale image
    """
    gray_img = np.dot(img[:,:,0:3], [0.299, 0.587, 0.114])
    return gray_img.astype(np.uint8)

def rgb_to_sepia(img):
    """
    Convert an RGB image to sepia.

    Parameters
    ----------
    img : numpy.ndarray
        Original image as a numpy array

    Returns
    -------
    numpy.ndarray
        Sepia-toned image
    """
    # Check if the image is RGBA (with an alpha channel)
    if img.ndim == 3 and img.shape[2] == 4:
        img = img[:, :, :3]  # Take only the first 3 channels (RGB)
    
    # Ensure image is an RGB image
    if img.ndim != 3 or img.shape[2] != 3:
        raise ValueError("Input image must be an RGB image.")

    sepia_filter = np.array([[0.393, 0.769, 0.189],
                             [0.349, 0.686, 0.168],
                             [0.272, 0.534, 0.131]])

    # Apply sepia filter
    sepia_img = np.dot(img, sepia_filter.T)

    # Clip values to the range [0, 255]
    sepia_img = np.clip(sepia_img, a_min=0, a_max=255).astype(np.uint8)

    return sepia_img

def blur_image(img, kernel_size=3):
    """
    Blur the image using an averaging kernel.

    Parameters
    ----------
    img : numpy.ndarray
        Original image as a numpy array
    kernel_size : int
        Size of the blurring kernel. Default is 3.

    Returns
    -------
    numpy.ndarray
        Blurred image
    """
    if kernel_size % 2 == 0:
        raise ValueError("Kernel size should be an odd number.")
    
    # Create an averaging kernel
    kernel = np.ones((kernel_size, kernel_size)) / (kernel_size * kernel_size)
    
    # Get image dimensions
    if img.ndim == 3:
        h, w, c = img.shape
        padded_img = np.pad(img, ((kernel_size//2, kernel_size//2), (kernel_size//2, kernel_size//2), (0, 0)), mode='reflect')
        blurred_img = np.zeros_like(img)
        
        for i in range(h):
            for j in range(w):
                for k in range(c):
                    blurred_img[i, j, k] = np.sum(padded_img[i:i+kernel_size, j:j+kernel_size, k] * kernel)
    else:
        h, w = img.shape
        padded_img = np.pad(img, ((kernel_size//2, kernel_size//2), (kernel_size//2, kernel_size//2)), mode='reflect')
        blurred_img = np.zeros_like(img)
        
        for i in range(h):
            for j in range(w):
                blurred_img[i, j] = np.sum(padded_img[i:i+kernel_size, j:j+kernel_size] * kernel)
    
    return blurred_img


def sharpen_image(image):
    """
    Sharpen the image using a sharpening kernel.

    Parameters
    ----------
    img : numpy.ndarray
        Original image as a numpy array.

    Returns
    -------
    numpy.ndarray
        Sharpened image.
    """
    # Define the sharpening kernel
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])

    if image.ndim == 3:  # RGB image
        # Handle each channel separately
        sharpened_image = np.zeros_like(image)
        for c in range(image.shape[2]):
            sharpened_image[:, :, c] = sharpen_image(image[:, :, c])
        return sharpened_image
    else:  # Grayscale image
        # Get image dimensions
        img_height, img_width = image.shape

        # Pad the image to handle border pixels
        padded_image = np.pad(image, pad_width=1, mode='constant', constant_values=0)

        # Initialize output image
        sharpened_image = np.zeros_like(image)

        # Convolve the image with the kernel
        for i in range(img_height):
            for j in range(img_width):
                # Extract the region of interest
                region = padded_image[i:i+3, j:j+3]
                # Apply the kernel
                sharpened_pixel = np.sum(region * kernel)
                # Clip the values to the range [0, 255] for image display
                sharpened_image[i, j] = np.clip(sharpened_pixel, 0, 255)

        return sharpened_image
    
def crop_center(img, crop_size):
    """
    Crop the image to a specified size from the center.

    Parameters
    ----------
    img : numpy.ndarray
        Original image as a numpy array.
    crop_size : tuple of int
        Size of the crop (height, width).

    Returns
    -------
    numpy.ndarray
        Cropped image.
    """
    h, w = img.shape[:2]
    new_h, new_w = crop_size

    # Adjust crop size if it's larger than the image size
    if new_h > h:
        new_h = h
    if new_w > w:
        new_w = w

    start_h = (h - new_h) // 2
    start_w = (w - new_w) // 2

    if img.ndim == 3:  # Color image
        return img[start_h:start_h+new_h, start_w:start_w+new_w, :]
    else:  # Grayscale image
        return img[start_h:start_h+new_h, start_w:start_w+new_w]


def crop_circle(img, center=None, radius=None):
    """
    Crop the image to a circular frame.

    Parameters
    ----------
    img : numpy.ndarray
        Original image as a numpy array.
    center : tuple of int, optional
        Center of the circular crop (x, y). Default is the center of the image.
    radius : int, optional
        Radius of the circular crop. Default is the smallest dimension divided by 2.

    Returns
    -------
    numpy.ndarray
        Circularly cropped image with the background set to black.
    """
    h, w = img.shape[:2]

    if center is None:
        center = (w // 2, h // 2)

    if radius is None:
        radius = min(center[0], center[1], w - center[0], h - center[1])

    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y - center[1])**2)
    
    mask = dist_from_center <= radius

    if img.ndim == 3:  # Color image
        cropped_img = np.zeros_like(img)  # Initialize with black (0,0,0)
        for c in range(img.shape[2]):  # Apply mask to each channel
            cropped_img[..., c] = np.where(mask, img[..., c], 0)
    else:  # Grayscale image
        cropped_img = np.zeros_like(img)  # Initialize with black (0)
        cropped_img[mask] = img[mask]  # Copy the pixels within the circular area

    return cropped_img

def crop_two_ellipses(img, angle1=None, angle2=None):
    """
    Crop the image to two overlapping elliptical frames that stand on the diagonals.

    Parameters
    ----------
    img : numpy.ndarray
        Original image as a numpy array.
    angle1 : float, optional
        Rotation angle of the first ellipse in degrees. Default is 45.
    angle2 : float, optional
        Rotation angle of the second ellipse in degrees. Default is -45.

    Returns
    -------
    numpy.ndarray
        Elliptically cropped image with the background set to black.
    """
    h, w = img.shape[:2]

    # Calculate k as half the diagonal of the square fitting within the image
    k = min(w, h) / np.sqrt(2)

    # Define semi-major and semi-minor axes ensuring a^2 + b^2 = k^2
    # We will choose a reasonable a and calculate b such that a != b
    a = k / 2  # Example value for a, this can be adjusted
    b = np.sqrt(k**2 - a**2)

    if angle1 is None:
        angle1 = 45
    if angle2 is None:
        angle2 = -45

    # Define the center of the ellipses (center of the image)
    center = (w // 2, h // 2)

    # Create meshgrid for the image coordinates
    Y, X = np.ogrid[:h, :w]

    # Create masks for the ellipses
    mask1 = (((X - center[0]) * np.cos(np.radians(angle1)) + (Y - center[1]) * np.sin(np.radians(angle1)))**2 / a**2 +
             ((X - center[0]) * np.sin(np.radians(angle1)) - (Y - center[1]) * np.cos(np.radians(angle1)))**2 / b**2) <= 1

    mask2 = (((X - center[0]) * np.cos(np.radians(angle2)) + (Y - center[1]) * np.sin(np.radians(angle2)))**2 / a**2 +
             ((X - center[0]) * np.sin(np.radians(angle2)) - (Y - center[1]) * np.cos(np.radians(angle2)))**2 / b**2) <= 1

    # Combine masks for the ellipses
    mask = mask1 | mask2

    # Create the cropped image with the background set to black
    if img.ndim == 3:  # Color image
        cropped_img = np.zeros_like(img)  # Initialize with black (0,0,0)
        for i in range(img.shape[2]):  # Apply the mask to each channel
            cropped_img[..., i] = np.where(mask, img[..., i], 0)
    else:  # Grayscale image
        cropped_img = np.zeros_like(img)  # Initialize with black (0)
        cropped_img[mask] = img[mask]  # Copy the pixels within the elliptical areas

    return cropped_img

def zoom_image(img, zoom_factor=2):
    """
    Zoom in or out of an image by a specified factor.

    Parameters
    ----------
    img : numpy.ndarray
        Original image as a numpy array.
    zoom_factor : int, optional
        Factor by which to zoom the image. Positive value to zoom in, negative to zoom out. Default is 2.

    Returns
    -------
    numpy.ndarray
        Zoomed image.
    """
    h, w = img.shape[:2]

    if zoom_factor > 0:
        # Zoom in
        zoomed_img = np.zeros((h * zoom_factor, w * zoom_factor, *img.shape[2:]), dtype=img.dtype)
        for i in range(h * zoom_factor):
            for j in range(w * zoom_factor):
                zoomed_img[i, j] = img[i // zoom_factor, j // zoom_factor]
    else:
        # Zoom out
        zoom_factor = abs(zoom_factor)
        zoomed_img = img[::zoom_factor, ::zoom_factor]

    return zoomed_img

**Main function**

In [ ]:
def main():
    img_path = input("Enter the image file path: ")

    # Load the image
    img = read_img(img_path)

    while True:
        print("\nSelect an operation:")
        print("1. Change Brightness")
        print("2. Change Contrast")
        print("3. Convert to Grayscale/Sepia")
        print("4. Flip Image")
        print("5. Blur/Sharpen Image")
        print("6. Crop Center")
        print("7. Crop Circle/Two Ellipses")
        print("8. Zoom In/Out")
        print("0. Perform All Operations")
        
        choice = int(input("Enter your choice (0-8): "))

        if choice == 0:
            # Perform all operations
            brightness_value = int(input("Enter brightness value: "))
            contrast_factor = float(input("Enter contrast factor: "))
            blur_kernel_size = int(input("Enter kernel size for blurring (odd number): "))
            crop_size_center = tuple(map(int, input("Enter crop size (height width): ").split()))
            zoomFactor = int(input("Enter zoom factor (positive to zoom in, negative to zoom out): "))

            operations = {
                "blur": lambda img: blur_image(img, blur_kernel_size),
                "sharpen": sharpen_image,
                "brightness": lambda img: change_brightness(img, brightness_value), # Ex: 20
                "contrast": lambda img: change_contrast(img, contrast_factor), # Ex: 1.5
                "grayscale": rgb_to_grayscale,
                "sepia": rgb_to_sepia,
                "flip_horizontal": flip_horizontal,
                "flip_vertical": flip_vertical,
                "crop_center": lambda img: crop_center(img, crop_size_center), # Ex: (200,100)
                "crop_circle": lambda img: crop_circle(img),
                "crop_two_ellipses": crop_two_ellipses,
                "zoom": lambda img: zoom_image(img, zoomFactor)
            }

            for name, func in operations.items():
                result_img = func(img)
                save_img(result_img, f"{img_path.split('.')[0]}_{name}.png")
                print(f"Saved {name} image as {img_path.split('.')[0]}_{name}.png")

        elif choice == 1:
            value = int(input("Enter brightness value: "))
            img_result = change_brightness(img, value)
            save_img(img_result, f"{img_path.split('.')[0]}_brightness.png")
            print(f"Saved brightness image as {img_path.split('.')[0]}_brightness.png")
        
        elif choice == 2:
            factor = float(input("Enter contrast factor: "))
            img_result = change_contrast(img, factor)
            save_img(img_result, f"{img_path.split('.')[0]}_contrast.png")
            print(f"Saved contrast image as {img_path.split('.')[0]}_contrast.png")

        elif choice == 3:
            img_grayscale = rgb_to_grayscale(img)
            save_img(img_grayscale, f"{img_path.split('.')[0]}_grayscale.png")
            print(f"Saved grayscale image as {img_path.split('.')[0]}_grayscale.png")

            img_sepia = rgb_to_sepia(img)
            save_img(img_sepia, f"{img_path.split('.')[0]}_sepia.png")
            print(f"Saved sepia image as {img_path.split('.')[0]}_sepia.png")

        elif choice == 4:
            flip_choice = int(input("Flip horizontally (1) or vertically (2): "))
            if flip_choice == 1:
                img_result = flip_horizontal(img)
                save_img(img_result, f"{img_path.split('.')[0]}_flip_horizontal.png")
                print(f"Saved flipped horizontally image as {img_path.split('.')[0]}_flip_horizontal.png")
            elif flip_choice == 2:
                img_result = flip_vertical(img)
                save_img(img_result, f"{img_path.split('.')[0]}_flip_vertical.png")
                print(f"Saved flipped vertically image as {img_path.split('.')[0]}_flip_vertical.png")
            else:
                print("Invalid choice.")
        
        elif choice == 5:
            kernel_size = int(input("Enter kernel size for blurring (odd number): "))
            img_blur = blur_image(img, kernel_size)
            save_img(img_blur, f"{img_path.split('.')[0]}_blur.png")
            print(f"Saved blurred image as {img_path.split('.')[0]}_blur.png")

            img_sharpen = sharpen_image(img)
            save_img(img_sharpen, f"{img_path.split('.')[0]}_sharpened.png")
            print(f"Saved sharpened image as {img_path.split('.')[0]}_sharpened.png")
        
        elif choice == 6:
            crop_size = tuple(map(int, input("Enter crop size (height width): ").split()))
            img_result = crop_center(img, crop_size)
            save_img(img_result, f"{img_path.split('.')[0]}_crop_center.png")
            print(f"Saved cropped center image as {img_path.split('.')[0]}_crop_center.png")

        elif choice == 7:
            img_circle= crop_circle(img)
            save_img(img_circle, f"{img_path.split('.')[0]}_crop_circle.png")
            print(f"Saved cropped circle image as {img_path.split('.')[0]}_crop_circle.png")

            angle1 = int(input("Enter angle for first ellipse: "))
            angle2 = int(input("Enter angle for second ellipse: "))
            img_ellipse = crop_two_ellipses(img, angle1, angle2)
            save_img(img_ellipse, f"{img_path.split('.')[0]}_crop_two_ellipses.png")
            print(f"Saved cropped two ellipses image as {img_path.split('.')[0]}_crop_two_ellipses.png")

        elif choice == 8:
            zoom_factor = int(input("Enter zoom factor (positive to zoom in, negative to zoom out): "))
            img_result = zoom_image(img, zoom_factor)
            save_img(img_result, f"{img_path.split('.')[0]}_zoom.png")
            print(f"Saved zoomed image as {img_path.split('.')[0]}_zoom.png")

        else:
            print("Invalid choice. Please enter a number between 0 and 8.")

        if choice == 0:
            break

In [ ]:
main()


Select an operation:
1. Change Brightness
2. Change Contrast
3. Convert to Grayscale/Sepia
4. Flip Image
5. Blur/Sharpen Image
6. Crop Center
7. Crop Circle/Two Ellipses
8. Zoom In/Out
0. Perform All Operations
Saved blur image as image_blur.png
Saved sharpen image as image_sharpen.png
Saved brightness image as image_brightness.png
Saved contrast image as image_contrast.png
Saved grayscale image as image_grayscale.png
Saved sepia image as image_sepia.png
Saved flip_horizontal image as image_flip_horizontal.png
Saved flip_vertical image as image_flip_vertical.png
Saved crop_center image as image_crop_center.png
Saved crop_circle image as image_crop_circle.png
Saved crop_two_ellipses image as image_crop_two_ellipses.png
Saved zoom image as image_zoom.png
